In [1]:
# classification
from __future__ import print_function, division
import torch
import numpy as np
import torch.optim as optim
import data
from torch.utils.data import DataLoader
from torch.autograd import Variable
from collections import OrderedDict
import csvgenerator as csvg
import os
import matplotlib.pyplot as plt
from torchvision import transforms, utils, models
from model import survresnet
import torch.nn as nn

In [2]:
# print gpu
torch.cuda.set_device(7)
currentDevice = torch.cuda.current_device()
print("Current GPU: " + str(currentDevice))
print(str(torch.cuda.device_count()))
print(str(torch.cuda.get_device_capability(currentDevice)))
print(torch.__version__)

USE_GPU = 1
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda:7')
else:
    device = "cpu"
print(device)

Current GPU: 7
8
(6, 1)
1.0.0
cuda:7


In [3]:
def new_data():   
    csvFilePath = csvg.get_csv_path()
    print(csvFilePath)
    transformed_dataset = data.HE_Dataset(csv_file=csvFilePath,
                                                   transform=data.Compose([                                                                                  
                                                   data.ToTensor()
                                               ]))
    # TODO: change the normalization parameters

    # batchsize 32->16
    dataloader = DataLoader(transformed_dataset, batch_size=30,
                            shuffle=True, num_workers=0)
    print(len(dataloader))
    return dataloader

In [4]:
cwd = os.getcwd()
path = os.path.join(cwd, 'Saved model', 'surclassifier.pth')

state_dict = torch.load(path, map_location=torch.device('cuda:7'))
model = survresnet()
model.load_state_dict(state_dict)
model.eval()
model.to(device)

survresnet(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Con

In [5]:
def test(trained_model, dataloader, num_batch=9):
    TP = 0
    FN = 0
    FP = 0
    TN = 0
    Tensor = torch.cuda.LongTensor
    with torch.no_grad():
        for iteration, batch in enumerate(dataloader):
            img_ser, img_dec = batch['input_sur'].float(), batch['input_dec'].float()
            img_ser = img_ser[0:3]
            inputs = torch.cat([img_ser, img_dec], dim=0).to(device) # three survival each batch
            ser_1 = Tensor(np.zeros(img_ser.size(0)))
            dec_1 = Tensor(np.ones(img_dec.size(0)))
            ser_2 = Tensor(np.ones(img_ser.size(0)))
            dec_2 = Tensor(np.zeros(img_dec.size(0)))
            label_temp_ser = torch.cat([ser_1, dec_1], dim=0).unsqueeze(0)
            label_temp_dec = torch.cat([ser_2, dec_2], dim=0).unsqueeze(0)
            labels = torch.cat([label_temp_ser, label_temp_dec], dim=0).transpose(1, 0) 
            labels = Variable(labels, requires_grad=False) # ser:1, dec:0
            predictions = torch.max(model(inputs), 1)[1].to('cpu')
            labels = torch.max(labels, 1)[1].to('cpu')
            predictions = predictions.numpy()
#             print(predictions)
            labels = labels.numpy()
            results = predictions==labels
            ser_results = results[0:3]
            dec_results = results[-30:]
            TP = np.sum(ser_results==True) + TP
            FN = np.sum(ser_results==False) + FN
            FP = np.sum(dec_results==False) + FP
            TN = np.sum(dec_results==True) + TN
#             print(ser_results)
#             print(dec_results)
            if iteration == num_batch:
                break
    accuracy = (TP + TN) / (TP + FN + FP + TN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN) 
    print("Accuracy: {:.4f}%, Precision: {:.4f}%, Recall: {:.4f}%".format(accuracy*100, precision*100, recall*100))

In [7]:
test(model, new_data(), 100)

/deepdata/adib/prognostic_study/pytorch_code/image_files.csv
39551
Accuracy: 98.9199%, Precision: 90.8257%, Recall: 98.0198%
